In [1]:
import sys
import time
import numpy as np
import pickle
import importlib

In [2]:
from qiskit import Aer, IBMQ
from qiskit.ignis.mitigation.measurement import TensoredMeasFitter

In [3]:
max_size = 21
max_length = 20

In [4]:
with open("pkls/raw_hist_list.pkl", "rb") as f:
    raw_hist_list = pickle.load(f)
with open("pkls/results_meas_cal.pkl", "rb") as f:
    results_meas_cal = pickle.load(f)

In [5]:
import libs_qrem
importlib.reload(libs_qrem)
from libs_qrem import IgnisFilter

In [6]:
xz_ignis_mitigator_list = []
for n in range(2, max_size + 1): # [2, ..., 57]
    mit_pattern = [[i] for i in range(n)]
    meas_fitter = TensoredMeasFitter(results_meas_cal[n - 2], mit_pattern=mit_pattern)
    xz_ignis_mitigator_list.append(meas_fitter.filter)
    if n % 10 == 0:
        print("size", n, "finished")
print("length of xz_ignis_mitigator_list: ", len(xz_ignis_mitigator_list))

size 10 finished
size 20 finished
length of xz_ignis_mitigator_list:  20


In [7]:
zx_ignis_mitigator_list = []
for n in range(2, max_size + 1): # [2, ..., 57]
    mit_pattern = [[i] for i in range(n)]
    meas_fitter = TensoredMeasFitter(results_meas_cal[n - 2], mit_pattern=mit_pattern)
    zx_ignis_mitigator_list.append(meas_fitter.filter)
    if n % 10 == 0:
        print("size", n, "finished")
print("length of zx_ignis_mitigator_list: ", len(zx_ignis_mitigator_list))

size 10 finished
size 20 finished
length of zx_ignis_mitigator_list:  20


In [8]:
ignis_hist_list = []
xz_ignis_total_times = []
zx_ignis_total_times = []
for i in range(9):
    print(i + 2, "qubits")
    
    t1 = time.perf_counter()
    ignis_hist_list.append( xz_ignis_mitigator_list[i].apply(raw_hist_list[2 * i]) )
    t2 = time.perf_counter()
    xz_ignis_total_times.append(t2 - t1)
    print("xz", t2 - t1, "s")
    
    t3 = time.perf_counter()
    ignis_hist_list.append( zx_ignis_mitigator_list[i].apply(raw_hist_list[2 * i + 1]) )
    t4 = time.perf_counter()
    zx_ignis_total_times.append(t4 - t3)
    print("zx", t4 - t3, "s")
    
    print()

2 qubits
xz 0.008840559999999442 s
zx 0.008218791999999198 s

3 qubits
xz 0.027968579999999577 s
zx 0.022867747999999466 s

4 qubits
xz 0.08280558100000057 s
zx 0.07839543399999904 s

5 qubits
xz 0.35770288400000005 s
zx 0.35385574200000036 s

6 qubits
xz 1.3830777659999995 s
zx 1.5527928689999992 s

7 qubits
xz 6.076807482000003 s
zx 6.453193708000001 s

8 qubits


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Users/yangbo/opt/anaconda3/envs/qiskitenv/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3444, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/vr/d82vcp297v78vk4sb0_nzzdc0000gn/T/ipykernel_10248/1250196096.py", line 8, in <module>
    ignis_hist_list.append( xz_ignis_mitigator_list[i].apply(raw_hist_list[2 * i]) )
  File "/Users/yangbo/opt/anaconda3/envs/qiskitenv/lib/python3.9/site-packages/qiskit/ignis/mitigation/measurement/filters.py", line 448, in apply
    res = minimize(fun, x0, method='SLSQP',
  File "/Users/yangbo/opt/anaconda3/envs/qiskitenv/lib/python3.9/site-packages/scipy/optimize/_minimize.py", line 631, in minimize
    return _minimize_slsqp(fun, x0, args, jac, bounds,
  File "/Users/yangbo/opt/anaconda3/envs/qiskitenv/lib/python3.9/site-packages/scipy/optimize/slsqp.py", line 437, in _minimize_slsqp
    g = append(wrapped_grad(x), 0.0)
  File "/Users/yangbo/opt/anacon

TypeError: object of type 'NoneType' has no len()

In [9]:
with open("./pkls/ignis_hist_list.pkl", "wb") as f:
    pickle.dump(ignis_hist_list, f)

In [10]:
with open("./pkls/ignis_total_times.pkl", "wb") as f:
    pickle.dump({"xz": xz_ignis_total_times, "zx": zx_ignis_total_times}, f)